# Semantic Kernel

Semantic Kernel can **`automatically orchestrate` different plugins by using a `planner`**. With the planner, a user can ask your application to achieve a complex goal. For example, if you have a function that identifies which animal is in a picture and another function that tells knock-knock jokes, your user can say, “Tell me a knock-knock joke about the animal in the picture in this URL,” and the planner will automatically understand that it needs to call the identification function first and the “tell joke” function after it. **Semantic Kernel will automatically search and combine your plugins to achieve that goal and create a plan.** Then, Semantic Kernel will execute that plan and provide a response to the user:

<img src="../assets/semantic-kernel-architecture.png" />


**`LLM Cascade:`** The process of sending simpler requests to simpler models and complex requests to more complex models.

In [10]:
import semantic_kernel as sk

kernel = sk.Kernel()
print(help(request=kernel))

Help on Kernel in module semantic_kernel.kernel object:

class Kernel(semantic_kernel.filters.kernel_filters_extension.KernelFilterExtension, semantic_kernel.functions.kernel_function_extension.KernelFunctionExtension, semantic_kernel.services.kernel_services_extension.KernelServicesExtension, semantic_kernel.reliability.kernel_reliability_extension.KernelReliabilityExtension)
 |  Kernel(plugins: semantic_kernel.functions.kernel_plugin.KernelPlugin | dict[str, semantic_kernel.functions.kernel_plugin.KernelPlugin] | list[semantic_kernel.functions.kernel_plugin.KernelPlugin] | None = None, services: Union[~AI_SERVICE_CLIENT_TYPE, list[~AI_SERVICE_CLIENT_TYPE], dict[str, ~AI_SERVICE_CLIENT_TYPE], NoneType] = None, ai_service_selector: semantic_kernel.services.ai_service_selector.AIServiceSelector | None = None, *, retry_mechanism: semantic_kernel.reliability.retry_mechanism_base.RetryMechanismBase = None, function_invocation_filters: list[tuple[int, collections.abc.Callable[[~FILTER_CONTE

In [11]:
print(dir(kernel))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_validator__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_r

In [12]:
from dotenv import load_dotenv

load_dotenv()
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

gpt35 = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo")
gpt4o = OpenAIChatCompletion(ai_model_id="gpt-4o")

kernel.add_service(service=gpt35)
kernel.add_service(service=gpt4o)

To send the prompt to the service, we need to use a method called **`create_semantic_function`**.

### Running a simple prompt

1. **Load the prompt in a string variable.**

In [13]:
prompt = "Finish the following knock-knock joke. Knock, knock. Who's there? Dishes. Dishes who?"

2. Create a *`function`* by using the **`add_function`** method of the kernel. The *`function_name`* and *`plugin_name`* parameters of the function are required but are not used, so you can give your function and plugin whatever name you want.

In [14]:
prompt_function = kernel.add_function(
    function_name="subrata",
    plugin_name="sample",
    prompt=prompt,
)

3. Call the function. Note that all invocation methods are *`Asynchronous`*, so we need to use *`await`* to wait for their return.

In [15]:
response = await kernel.invoke(function=prompt_function, request=prompt)

print(response)

Dishes the police, open up!


In [16]:
# Entire Code in One Place
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Initialize the Kernel
kernel = sk.Kernel()

# Define Connectors
gpt35 = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo")
gpt4o = OpenAIChatCompletion(ai_model_id="gpt-4o")

# Add the connectors to the kernel's service
kernel.add_service(service=gpt35)
kernel.add_service(service=gpt4o)

# Write a prompt
prompt = "Finish the following knock-knock joke. Knock, knock. Who's there? Dishes. Dishes who?"

# Create a function (semantic) using add_function method of the kernel
prompt_function = kernel.add_function(
    function_name="Subrata",
    plugin_name="Mondal",
    prompt=prompt,
)

# Invoke the created function
response = await kernel.invoke(
    function=prompt_function,
    request=prompt,
)

print(response)


Dishes the police, open up!


## Semantic Functions and Native Functions

- **Semantic Functions:** are functions that connect to AI Services (LLM) to perform a task. The default parameter for all semantic functions is called *`input`*.

- **Native Functions:** are regular functions written in your programming language (Python).


The reason to differentiate a Native Function from any other Regular Function in your code is that the native funcion will have additional attributes that will tell the Kernel what it does.

After loading a native function into the Kernel, you can use it in chains that combine native and semantic functions. On top of it, **`Semantic Kernel Planner`** can use the function when creating a plan to achieve a user goal.

### Modified Semantic Function in Python

1. **Adding parameter to the Semantic Function**

In [17]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

args = KernelArguments(input="Boo")

response = await kernel.invoke(
    function=prompt_function,
    request=prompt,
    arguments=args,
)

print(response)

Dishes the police! Open up!


### Create a Native Function

Native Functions are created in the same language our application is using like Python. 

In Python, the **Native functions need to be inside a `Class`**. The Class used to be called **`Skill`** and now it's called **`Plugin`**.

- **`Plugin:`** is just a *`collection of functions`*. You **cannot mix both the `Native` and `Semantic Function` in the same Plugin/Skill**. For, example we'll create a **`ShowManager` Plugin**.

**How to create a Native Function?**

To create a Native Function, we will use **`@kernel_function` decorator**. And the decorator must contain fields for **`description`** and **`name`**.

In [19]:
import random
from semantic_kernel.functions.kernel_function_decorator import kernel_function


class ShowManager:
    @kernel_function(
        description="Randomly choose among a theme for a joke.",
        name="random_theme",
    )
    def random_theme(self) -> str:
        themes: list[str] = ["Boo", "Dishes", "Art", "Needle", "Tank", "Police"]
        theme: str = random.choice(seq=themes)
        return theme


Now, to load the *Plugin (ShowManager)* and all its *functions* in the Kernel, we use the **`add_plugin`** method of the Kernel.

In [21]:
theme_choice_kernel_plugin = kernel.add_plugin(
    plugin=ShowManager(), plugin_name="ShowManager"
)

To call the Native Function from a *Plugin (ShowManager)*, simply put the name of the method within brackets.

In [27]:
response = await kernel.invoke(
    theme_choice_kernel_plugin["random_theme"],
)

print(response)

Art
